In [1]:
!pip list | findstr torch

torch                  2.3.1
torchaudio             2.3.1
torchvision            0.18.1


In [2]:
from IPython import display
display.clear_output()

In [3]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.83  Python-3.11.9 torch-2.3.1 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
Setup complete  (12 CPUs, 23.7 GB RAM, 327.4/475.3 GB disk)


In [4]:
!nvidia-smi

Fri Aug 30 20:15:29 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 556.12                 Driver Version: 556.12         CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 3050 ...  WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   65C    P8              3W /   35W |       0MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [5]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [6]:
import torch

# Set device manually
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Use device for operations
torch.zeros(1).to(device)

Device: cuda


tensor([0.], device='cuda:0')

In [7]:
import torch
torch.cuda.is_available()

True

In [8]:
import cv2
import numpy as np
from ultralytics import YOLO
import threading

In [9]:
model = YOLO(r"C:\Users\Arjun Kharbanda\Documents\SIH-2\best-2.pt")

In [14]:
from ultralytics import YOLO

# Load the pre-trained YOLO model
model = YOLO(r"C:\Users\Arjun Kharbanda\Documents\SIH-2\best.pt")  # Replace with the path to your model

# Predict without showing the images
results = model.predict(source="0", show=False, conf=0.5)  # Use source=0 for webcam input

# Process results (e.g., print or save them)
for result in results:
    print(result)



1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

0: 480x640 1 water, 74.8ms
0: 480x640 1 water, 23.0ms
0: 480x640 1 water, 19.7ms


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1367: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvWaitKey'


In [9]:
import cv2
import time
from ultralytics import YOLO


# Part 2: Run the YOLO detection script

# Load the pre-trained YOLO model
model = YOLO(r"C:\Users\Arjun Kharbanda\Documents\SIH-2\best-2.pt")  # Replace with the path to your model

# Define the hand sign labels that the model can detect
labels = ["Hello", "Help", "Home", "No", "Please", "Yes"]

# Initialize variables for detection timing
detected_start_time = None
min_detection_duration = 3  # Minimum duration for detection in seconds
detected_label = None

# Predict with showing the images (adjusted to display bounding boxes)
results = model.predict(source="0", show=True, conf=0.3)  # Use source=0 for webcam input

# Loop through the results and draw bounding boxes for detected hand signs
for result in results:
    # Get the original image
    img = result.orig_img
    current_time = time.time()
    detected_this_frame = False

    # Loop through the detections in the current frame
    for det in result.boxes.data:
        # Get the label and confidence score for the detected object
        class_id = int(det.cls[0])
        label = labels[class_id]  # Get the label from the predefined list
        confidence = det.conf[0]

        # Check if the detected object is one of the specified hand signs
        if label in labels:
            detected_this_frame = True
            if detected_start_time is None:
                detected_start_time = current_time
                detected_label = label

            # Extract bounding box coordinates (x1, y1, x2, y2)
            x1, y1, x2, y2 = map(int, det.xyxy[0])

            # Draw the bounding box
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green box with thickness 2

            # Put the label and confidence on the bounding box
            cv2.putText(img, f"{label} {confidence:.2f}", (x1, y1 - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)  # Green text with thickness 2

    # Check if a hand sign has been detected continuously for the required duration
    if detected_this_frame:
        if detected_start_time and (current_time - detected_start_time) >= min_detection_duration:
            print(f"Detected {detected_label} continuously for {min_detection_duration} seconds")
            # Store or process detected data here if needed
            detected_start_time = None  # Reset detection start time after storing data
    else:
        detected_start_time = None  # Reset detection start time if no detection in this frame

    # Display the image with bounding boxes
    cv2.imshow("Hand Sign Detection", img)

    # Exit the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release resources
cv2.destroyAllWindows()


WARNING  Environment does not support cv2.imshow() or PIL Image.show()
OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1301: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'


1/1: 0... Success  (inf frames of shape 640x480 at 30.00 FPS)


WARNING  inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities f

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1367: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvWaitKey'
